In [1]:

from collections import OrderedDict
from functools import partial
import os
from pathlib import Path
from string import Template
from typing import cast, Callable, Generator, List, Tuple

from pytest import fixture  # type: ignore

from pyqir.generator import bitcode_to_ir, types  # type: ignore
from pyqir.generator import Builder, IntPredicate, Value  # type: ignore
from pytket import Circuit  # type: ignore
from pytket.circuit import (  # type: ignore
    CircBox,
    OpType,
)
from pytket.circuit.logic_exp import (  # type: ignore
    BitNot,
    if_bit,
    if_not_bit,
    reg_eq,
    reg_neq,
    reg_geq,
    reg_gt,
    reg_lt,
    reg_leq,
)

from pytket_qir.converter import Block, circuit_to_qir, write_qir_file  # type: ignore

from pytket_qir.gatesets.base import FuncName, FuncNat, FuncSpec  # type: ignore

from pytket_qir.gatesets.base import CustomGateSet, CustomQirGate
from pytket_qir.gatesets.pyqir import _TK_TO_PYQIR
from pytket_qir.generator import QirGenerator
from pytket_qir.module import Module



In [2]:


from pytket.circuit import Circuit



In [3]:
circ = Circuit(3)
circ.CRz(0.5, 0, 1)
circ.T(2)
circ.CSWAP(2, 0, 1)

[CRz(0.5) q[0], q[1]; T q[2]; CSWAP q[2], q[0], q[1]; ]

In [4]:
    circ = Circuit(2)
    a = circ.add_c_register("a", 3)
    b = circ.add_c_register("b", 3)
    c = circ.add_c_register("c", 3)
    circ.add_classicalexpbox_register(a & b, c)
    circ.add_classicalexpbox_register(a | b, c)
    circ.add_classicalexpbox_register(a ^ b, c)
    circ.add_classicalexpbox_register(a + b, c)
    circ.add_classicalexpbox_register(a - b, c)
    circ.add_classicalexpbox_register(a * b, c)
    # circ.add_classicalexpbox_register(a // b, c) No division yet.
    circ.add_classicalexpbox_register(a << b, c)
    circ.add_classicalexpbox_register(a >> b, c)
    circ.add_classicalexpbox_register(reg_eq(a, b), c)
    circ.add_classicalexpbox_register(reg_neq(a, b), c)
    circ.add_classicalexpbox_register(reg_gt(a, b), c)
    circ.add_classicalexpbox_register(reg_geq(a, b), c)
    circ.add_classicalexpbox_register(reg_lt(a, b), c)
    circ.add_classicalexpbox_register(reg_leq(a, b), c)

    module = Module(
        name="Generated from input pytket circuit", num_qubits=2, num_results=9
    )
    wasm_int_type = types.Int(32)
    qir_int_type = types.Int(32)
    qir_generator = QirGenerator(
        circuit=circ,
        module=module,
        wasm_int_type=wasm_int_type,
        qir_int_type=qir_int_type,
    )
    populated_module = qir_generator.circuit_to_module(
        qir_generator.circuit, qir_generator.module
    )

    with open("ClassicalCircuit-1.ll", "w") as output_file:
        output_file.write(populated_module.module.ir())

In [25]:
    circ = Circuit(3)
    a = circ.add_c_register("a", 3)
    qb = circ.add_q_register("qb", 3)
    circ.H(0)
    circ.H(2)
    circ.H(1)
    # circ.add_barrier([0, 1]) not working
    circ.X(0)
    circ.Y(1)
    circ.Z(1)
    circ.H(0, condition=a[0])
    circ.H(0)
    # circ.Measure(qb[0], a[0])
    cbcirc = Circuit(3)
    cbcirc.H(0) # , condition=a[0]
    
    circ_box = CircBox(cbcirc)
    # circ.add_circbox(circ_box, [0, 1, 2], condition=a[0])
    

    module = Module(
        name="Generated from input pytket circuit", num_qubits=3, num_results=9
    )
    wasm_int_type = types.Int(32)
    qir_int_type = types.Int(64)
    qir_generator = QirGenerator(
        circuit=circ,
        module=module,
        wasm_int_type=wasm_int_type,
        qir_int_type=qir_int_type,
    )
    populated_module = qir_generator.circuit_to_module(
        qir_generator.circuit, qir_generator.module
    )

    #with open("ClassicalCircuit-cond.ll", "w") as output_file:
    #    output_file.write(populated_module.module.ir())
    
    print(populated_module)
    print(dir(populated_module))
    print("\n\n")
    print(dir(populated_module.module.ir))
    print("\n\n")
    print(populated_module.module.ir.__str__())
    print(populated_module.module.ir())

found something conditional
OpType.H
[]
True
H
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'create', 'dagger', 'free_symbols', 'get_name', 'get_unitary', 'is_clifford_type', 'is_gate', 'n_qubits', 'params', 'transpose', 'type']
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_gateset', 'builder', 'gateset', 'module', 'qis', 'wasm_handler']



['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format_

thread '<unnamed>' panicked at 'Found PointerValue(PointerValue { ptr_value: Value { name: "", address: 0x2ab4b90, is_const: true, is_null: true, is_undef: false, llvm_value: "%Result* null", llvm_type: "%Result*" } }) but expected the IntValue variant', /usr/local/cargo/git/checkouts/inkwell-946411d814d2c9f8/6ab2b19/src/values/enums.rs:399:13


PanicException: Found PointerValue(PointerValue { ptr_value: Value { name: "", address: 0x2ab4b90, is_const: true, is_null: true, is_undef: false, llvm_value: "%Result* null", llvm_type: "%Result*" } }) but expected the IntValue variant